In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 

data = pd.read_csv("C:/Users/sovta/Downloads/archive/mnist_train.csv")
data.head()

MemoryError: 

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape


In [ ]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2
    


In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)


def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y)/Y.size
    
    
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for _ in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha) 
        if _ % 10 == 0:
            print(f'Iteration: {_}')
            print(f'Accuracy: {get_accuracy(get_predictions(A2), Y)}')
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10,  500)

Iteration: 0
[1 6 7 ... 7 6 6] [8 9 3 ... 2 9 4]
Accuracy: 0.07494915254237289
Iteration: 10
[1 6 5 ... 2 6 6] [8 9 3 ... 2 9 4]
Accuracy: 0.1338135593220339
Iteration: 20
[1 7 5 ... 3 7 7] [8 9 3 ... 2 9 4]
Accuracy: 0.20476271186440678
Iteration: 30
[1 7 5 ... 3 7 7] [8 9 3 ... 2 9 4]
Accuracy: 0.2873389830508475
Iteration: 40
[1 7 5 ... 8 7 7] [8 9 3 ... 2 9 4]
Accuracy: 0.37015254237288137
Iteration: 50
[1 7 5 ... 8 9 7] [8 9 3 ... 2 9 4]
Accuracy: 0.4317966101694915
Iteration: 60
[1 7 5 ... 8 9 4] [8 9 3 ... 2 9 4]
Accuracy: 0.47850847457627116
Iteration: 70
[1 9 5 ... 8 9 4] [8 9 3 ... 2 9 4]
Accuracy: 0.5159322033898305
Iteration: 80
[1 9 5 ... 8 9 4] [8 9 3 ... 2 9 4]
Accuracy: 0.5508813559322034
Iteration: 90
[1 9 5 ... 8 9 4] [8 9 3 ... 2 9 4]
Accuracy: 0.5837118644067797
Iteration: 100
[1 9 5 ... 8 9 4] [8 9 3 ... 2 9 4]
Accuracy: 0.613593220338983
Iteration: 110
[1 9 5 ... 8 9 4] [8 9 3 ... 2 9 4]
Accuracy: 0.6416271186440679
Iteration: 120
[8 9 5 ... 8 9 4] [8 9 3 ... 2 9 

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions



In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[3 7 3 9 7 1 7 8 0 7 8 5 8 0 6 5 5 1 6 7 4 6 4 3 8 8 7 1 9 6 5 1 9 6 9 9 6
 8 2 1 2 8 7 7 2 3 3 7 5 8 6 7 5 7 1 8 1 2 9 3 3 2 1 8 5 9 8 9 3 8 7 0 0 4
 3 4 7 4 5 5 9 3 1 4 6 8 3 2 2 6 7 3 6 7 0 9 9 7 7 2 3 1 7 0 2 1 6 9 6 6 4
 8 0 1 3 1 6 2 3 6 7 5 4 2 9 5 7 4 1 0 0 2 6 6 9 1 2 1 2 5 6 5 3 6 9 7 6 1
 4 7 6 3 0 5 5 1 3 9 4 7 8 0 5 0 1 1 7 1 7 7 4 8 5 5 7 4 1 7 6 2 4 1 2 6 6
 1 6 3 4 1 4 5 2 2 8 1 2 8 8 4 5 3 6 2 3 5 7 7 8 1 2 7 2 7 2 8 7 9 6 6 3 8
 7 2 0 1 2 8 5 1 4 1 5 3 8 8 1 4 0 9 4 5 1 9 2 0 3 5 8 3 9 3 8 6 6 5 2 4 6
 3 1 4 4 6 9 0 1 8 3 6 3 5 9 3 3 3 8 0 2 0 8 9 8 6 5 6 5 8 2 0 9 3 3 0 9 2
 5 0 0 8 5 7 9 1 1 3 7 7 5 9 1 4 1 7 7 2 2 6 9 1 4 9 8 0 2 3 1 1 4 2 4 6 4
 3 8 6 1 4 8 8 0 5 4 8 4 0 3 2 4 6 3 1 6 6 4 8 7 1 9 0 1 8 4 1 2 7 1 6 4 8
 1 9 6 8 1 7 0 8 8 3 0 2 2 3 6 3 7 8 6 4 7 4 3 4 8 2 0 3 7 0 0 6 8 9 2 2 1
 2 2 2 4 7 2 6 1 2 1 7 7 3 1 2 1 1 8 2 0 2 4 6 6 9 7 9 2 4 3 3 7 2 2 2 1 3
 1 2 8 9 2 7 3 9 2 2 5 4 3 4 9 6 9 6 0 2 2 4 0 1 8 1 8 1 9 6 3 6 9 4 1 2 7
 7 5 0 4 7 8 9 0 6 8 6 7 

0.858

In [ ]:
W_1, b_1, W_2, b_2 = gradient_descent(X_train, Y_train, 0.20,  500)

Iteration: 0
[6 5 8 ... 6 0 9] [8 9 3 ... 2 9 4]
Accuracy: 0.06830508474576272
Iteration: 10
[6 8 6 ... 8 2 9] [8 9 3 ... 2 9 4]
Accuracy: 0.16022033898305085
Iteration: 20
[8 8 6 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.2218813559322034
Iteration: 30
[8 2 9 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.3249491525423729
Iteration: 40
[8 0 9 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.39622033898305087
Iteration: 50
[8 2 8 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.4689491525423729
Iteration: 60
[8 2 8 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.5353559322033898
Iteration: 70
[8 2 8 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.5918474576271187
Iteration: 80
[8 2 8 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.6374576271186441
Iteration: 90
[8 2 5 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.6735254237288135
Iteration: 100
[8 2 5 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.7000169491525424
Iteration: 110
[8 2 5 ... 8 9 9] [8 9 3 ... 2 9 4]
Accuracy: 0.7197288135593221
Iteration: 120
[8 2 3 ... 8 9 9] [8 9 3 ... 2 9 

In [ ]:
dev_predictions_1 = make_predictions(X_dev, W_1, b_1, W_2, b_2)
get_accuracy(dev_predictions_1, Y_dev)

[3 7 3 4 7 8 7 8 0 7 8 5 8 0 6 5 5 1 6 7 4 6 4 3 8 5 7 1 9 6 5 1 9 6 7 9 6
 8 2 1 2 8 7 7 8 3 3 7 5 8 6 7 5 7 1 8 1 1 9 3 3 2 1 8 5 9 8 9 3 8 7 0 5 4
 3 4 7 4 5 5 9 3 1 4 6 8 3 1 2 6 7 3 6 7 0 9 9 7 7 2 3 1 7 0 2 1 6 9 6 6 4
 1 0 1 3 1 6 2 3 4 7 4 4 2 9 5 7 5 1 5 0 2 6 6 4 1 2 1 2 5 6 8 3 6 9 7 6 1
 9 7 6 3 0 5 5 2 3 9 4 7 8 0 3 0 8 1 7 1 7 7 4 8 5 5 7 4 1 7 6 2 4 1 2 6 2
 1 6 3 4 1 4 5 2 2 8 1 2 8 8 9 5 3 6 6 3 5 9 7 8 1 2 7 2 7 2 8 7 9 6 6 3 8
 7 2 0 1 2 8 5 1 4 1 5 3 6 8 1 4 0 9 4 4 1 9 2 0 8 5 8 3 9 3 8 2 6 9 2 4 6
 3 1 4 4 6 4 0 1 1 3 6 3 3 9 3 3 3 8 0 2 0 8 9 8 2 5 6 5 8 6 0 9 3 3 0 9 2
 5 0 0 8 5 7 9 1 1 3 7 9 5 9 1 4 1 7 7 2 3 6 9 1 4 9 8 0 2 3 1 1 4 2 9 6 4
 3 8 4 1 4 4 8 0 5 4 8 4 5 3 2 4 6 3 1 6 6 4 8 7 1 9 0 1 3 4 1 2 7 1 6 4 5
 1 9 6 8 1 7 0 8 5 3 0 2 2 3 6 3 7 8 6 4 7 4 3 4 8 7 0 3 7 0 0 6 8 3 2 2 1
 2 2 2 4 7 2 6 1 2 1 7 7 3 1 2 1 8 8 2 5 2 4 6 6 9 7 9 2 4 3 3 7 2 6 2 5 3
 1 2 8 9 6 7 3 9 6 0 5 4 3 4 9 6 9 6 0 2 2 4 0 3 8 1 8 1 9 6 3 4 9 4 1 2 7
 7 8 0 4 9 8 9 0 6 8 5 9 

0.886